In [77]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [78]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['PMT_Signal_100mW_07112017.txt', 'PMT_Signal_90mW_07112017.txt', 'PMT_Signal_80mW_07112017.txt', 'PMT_Signal_70mW_07112017.txt', 'PMT_Signal_60mW_07112017.txt', 'PMT_Signal_50mW_07112017.txt', 'PMT_Signal_40mW_07112017.txt', 'PMT_Signal_30mW_07112017.txt', 'PMT_Signal_20mW_07112017.txt', 'PMT_Signal_15mW_07112017.txt', 'PMT_Signal_Background_07112017.txt']


In [79]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [80]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [81]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (417*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay2 = []
    for i in range(0,len(x)):
        timedelay = -((x[i,0]*itr + offset)*25.4*2/(10**-12*c))+T1
        timeDelay2.append(timedelay)
    return(timeDelay2)

In [90]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,200):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(200,len(allFiles))
PDvalues = PDvalues.values.reshape(200,len(allFiles))

Background = PDvalues[:,-1]
M_Background = np.mean(Background)
Mean_PMT = PDvalues[:,0:10].mean(axis = 0) - M_Background
STD_PMT = PDvalues[:,0:10].std(axis = 0)
err1 = STD_PMT/(200**0.5)
print(err1)
print(Mean_PMT)
LaserPower = [100,90,80,70,60,50,40,30,20,15]
LogPMT = np.log10(np.absolute(Mean_PMT))
LogLaser = np.log10(LaserPower)



[ 0.00036179  0.00021817  0.00031715  0.00024183  0.00019902  0.00019243
  0.00021645  0.00021841  0.00022961  0.00022326]
[-0.169649  -0.134256  -0.099675  -0.078125  -0.059459  -0.0440795
 -0.030304  -0.015241  -0.006191  -0.0045075]


In [101]:
w=(err1/(np.absolute(Mean_PMT)*log(10)))**-2
Delta = sum(w)*sum(w*LogLaser**2) - (sum(w*LogLaser))**2
A = (sum(w*LogLaser**2)*sum(w*LogPMT) - sum(w*LogLaser)*sum(w*LogLaser*LogPMT))/Delta
B = (sum(w)*sum(w*LogLaser*LogPMT) - (sum(w*LogPMT)*sum(w*LogLaser)))/Delta
# Least Squares Fitting Parameters above
print(w)
print(A)
print(B)
# Errors in Least Squares Plotting
err = (((len(LogPMT)-2)**-1) * sum((LogPMT - A -B*LogLaser)**2)**0.5)
errorA = (sum(w*LogLaser**2)/Delta)**0.5
errorB = (sum(w)/Delta)**0.5
print(errorA)
print(errorB)

[ 1165816.26453837  2007768.98566776   523677.34501376   553356.82606449
   473248.76667941   278209.17648938   103928.24032528    25817.46120283
     3854.53718438     2161.16233746]
-4.71862815733
1.96767173636
0.00825043802733
0.00431958396217


In [106]:
plt.plot(LogLaser, B*LogLaser + A, 'k',label = 'Fit: y = 1.97x-4.71')
plt.plot(LogLaser,LogPMT,'b*',label = 'Data')
plt.plot()
plt.grid(True)
plt.xlabel('Log(Laser Power)')
plt.ylabel('Log(PMT Signal)')
plt.legend(loc = 'upper left')
#plt.show()
plt.savefig('PMTSignalVsLaserPower_07112017.png')



In [103]:
fit = np.polyfit(LogLaser,LogPMT,1,w=(err1/(np.absolute(Mean_PMT)*log(10)))*-2)
f = np.poly1d(fit)
pPower = np.linspace(LogLaser[1],LogLaser[-1],100)
f_new = f(pPower)
print(f_new)
print(f)

[-1.03429437 -1.0477679  -1.06124143 -1.07471496 -1.08818849 -1.10166202
 -1.11513555 -1.12860907 -1.1420826  -1.15555613 -1.16902966 -1.18250319
 -1.19597672 -1.20945025 -1.22292377 -1.2363973  -1.24987083 -1.26334436
 -1.27681789 -1.29029142 -1.30376495 -1.31723848 -1.330712   -1.34418553
 -1.35765906 -1.37113259 -1.38460612 -1.39807965 -1.41155318 -1.4250267
 -1.43850023 -1.45197376 -1.46544729 -1.47892082 -1.49239435 -1.50586788
 -1.51934141 -1.53281493 -1.54628846 -1.55976199 -1.57323552 -1.58670905
 -1.60018258 -1.61365611 -1.62712964 -1.64060316 -1.65407669 -1.66755022
 -1.68102375 -1.69449728 -1.70797081 -1.72144434 -1.73491786 -1.74839139
 -1.76186492 -1.77533845 -1.78881198 -1.80228551 -1.81575904 -1.82923257
 -1.84270609 -1.85617962 -1.86965315 -1.88312668 -1.89660021 -1.91007374
 -1.92354727 -1.9370208  -1.95049432 -1.96396785 -1.97744138 -1.99091491
 -2.00438844 -2.01786197 -2.0313355  -2.04480902 -2.05828255 -2.07175608
 -2.08522961 -2.09870314 -2.11217667 -2.1256502  -2.

[ 0.18523154  0.14114748  0.27637458  0.26886072  0.29072692  0.37917899
  0.62038774  1.24472447  3.22139423  4.30215745]
